In [14]:
import pandas as pd
import numpy as np
from scipy import sparse
# import stellargraph as sg

In [15]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [200]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import os
import networkx as nx
import numpy as np
import pandas as pd
import random
# from stellargraph import datasets
# from IPython.display import display, HTML
# from stellargraph.data import UniformRandomMetaPathWalk

%matplotlib inline

In [18]:
# sg.__version__

In [19]:
A_tr = sparse.load_npz('./data/A_tr.npz')
B_tr = sparse.load_npz('./data/B_tr.npz')
P_tr = sparse.load_npz('./data/P_tr.npz')
A_tr_csr = A_tr
A_tr_csc = A_tr.tocsc(copy=True)

In [20]:
A_tr.todense()

matrix([[1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]])

In [192]:
# new
def init_app():
    '''
    Initialize the first app
    '''
    numberOfNodes = A_tr.shape[0]
    appNumber = np.random.randint(numberOfNodes)
#     print('find App:', appNumber)
    return appNumber
    
def chose_idx(row):
    next_index = np.random.choice(np.nonzero(row)[1])
    return next_index
    
def A_api(appNumber):
    '''
    Find the next api using matrix A given app
    '''
    row = A_tr[appNumber]
    next_index = chose_idx(row)
    path.append('api_%d'%next_index)
    return next_index

def A_app(next_index):
    '''
    Find the next app using matrix A given api
    '''
    row = A_tr[:, next_index]
    next_index = np.random.choice(np.nonzero(row)[0])
    path.append('app_%d'%next_index)
    return next_index

def B_api(next_index):
    row = B_tr[next_index]
    next_index = chose_idx(row)
    path.append('api_%d'%next_index)
    return next_index

def P_api(next_index):
    row = P_tr[next_index]
    next_index = chose_idx(row)
    path.append('api_%d'%next_index)
    return next_index

def metapath2vec(path, path_l, next_index):
    # We have to sart with an app
    appNumber = init_app()
    path.append('app_%s'%appNumber)

    for i in metapaths:
        if(i == 'A'):
            if(path_l != 1):
                next_index = A_api(appNumber)
            else:
                next_index = A_app(next_index)

        if(i == 'B'):
            next_index = B_api(next_index)
        if(i == 'P'):
            next_index = P_api(next_index)        
        path_l -= 1
    return path

In [194]:
# old
def init_app():
    numberOfNodes = A_tr.shape[0]
#     print(numberOfNodes)
    appNumber = np.random.choice(range(numberOfNodes))
#     print('find App:', appNumber)
    return appNumber
    
def chose_idx(row):
    r = pd.Series(row.todense()[0].getA1())
    idx = r[r == 1].index.tolist()
    next_index = np.random.choice(idx)
    return next_index
    
def A_api(appNumber):
    row = A_tr[appNumber]
#     p = 1 / row.todense().sum()
    next_index = chose_idx(row)
    path.append('api_%d'%next_index)
    return next_index

def A_app(next_index):
    row = A_tr[:, next_index]
    r = pd.Series(row.todense().getA1())
    idx = r[r == 1].index.tolist()
    next_index = np.random.choice(idx)
    path.append('app_%d'%next_index)
    return next_index

def B_api(next_index):
    row = B_tr[next_index]
    next_index = chose_idx(row)
    path.append('api_%d'%next_index)
    return next_index

def P_api(next_index):
    row = P_tr[next_index]
    next_index = chose_idx(row)
    path.append('api_%d'%next_index)
    return next_index

def metapath2vec(path, path_l, next_index):
    # We have to sart with an app
    appNumber = init_app()
    path.append('app_%s'%appNumber)

    for i in metapaths:
    #     print(path_l != 1)
        if(i == 'A'):
            if(path_l != 1):
                next_index = A_api(appNumber)
            else:
                next_index = A_app(next_index)

        if(i == 'B'):
            next_index = B_api(next_index)
        if(i == 'P'):
            next_index = P_api(next_index)        
    #             print(row)
        path_l -= 1
    return path

In [187]:
metapaths = ["A", "B", "P", "B", "P", "B", "A"]

In [220]:
print('Using numpy')
%timeit np.random.choice(np.nonzero(A_tr[0])[1])
print('\nUsing random')

%timeit random.choice(np.nonzero(A_tr[0])[1])

Using numpy
238 µs ± 6.76 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

Using random
222 µs ± 8.63 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [219]:
%%time
# test one iteration of metapath2vec
for i in range(10):
    path = []
    path_l = len(metapaths)
    next_index = -1
    (metapath2vec(path, path_l, next_index))
# print(path)

CPU times: user 4.49 s, sys: 116 ms, total: 4.6 s
Wall time: 4.6 s


In [136]:
%%time
A_tr.shape[0]

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs


656

429

In [9]:
# %%time
# f = open("meta_ABPBPBA.cor", "w")

# for i in range(120000):
#     path = []
#     path_l = len(metapaths)
# #     print(path_l)
#     next_index = -1
#     f.write(' '.join(metapath2vec(path, path_l, next_index)) + '\n')
    
# f.close()

CPU times: user 14h 15min 35s, sys: 12min 44s, total: 14h 28min 20s
Wall time: 14h 29min 3s


In [10]:
from gensim import utils

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = 'meta_ABPBA.cor'
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield line.strip().split(' ')

In [11]:
%%time

import gensim.models

sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences, min_count=1, size=200)

/Users/jeffliao/.local/lib/python3.7/site-packages/gensim/models/base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


CPU times: user 2min 19s, sys: 35.3 s, total: 2min 55s
Wall time: 2min 14s


In [12]:
def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = []  # positions in vector space
    labels = []  # keep track of words to label our data again later
    for word in model.wv.vocab:
        if 'app' in word:
            if int(word.split('_')[1]) > 332:
                labels.append(1)
            else:
                labels.append(0)

            vectors.append(model.wv[word])
            # labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)

    # reduce using t-SNE
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='markers',
                       text=labels, marker=dict(size=5, color=labels))

    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))
    plt.show()
    
def cosine(u, v):
    return np.dot(u, v)/(np.linalg.norm(u)* np.linalg.norm(v))

In [13]:
x_vals, y_vals, labels = reduce_dimensions(model)
plot_with_plotly(x_vals, y_vals, labels)

In [ ]:
model